# Setup do Código

Aqui o espaço para setar todos as variáveis para importar planilha

In [1]:
PATH = './src'
CLIENT_PATH = f'{PATH}/Clientes'

portfolioCliente = 'Portfolio Global - KING.xlsx'
databaseFundos = 'New_query_2023_11_16 (1).xlsx'
historicoBenchmarks = 'arvore_classificacao_insper.xlsx'

<br>

---

<br>

# Rebalanceamento de Carteira

## Importando Bibliotecas

Bibliotecas Importantes usadas para o Código

In [2]:
import re
import locale
import numpy as np
import pandas as pd
import openpyxl as xl 
from skimage.measure import label, regionprops

pd.options.mode.chained_assignment = None  # default='warn'
locale.setlocale(locale.LC_ALL, 'pt-BR.utf-8')

'pt-BR.utf-8'

## Leitura dos Excels e Planilhas:

Função para ler e receber os nomes de cada planilha dentro do excel

In [3]:
# Recebe Caminho do Arquivo e Nome do Arquivo

def getSheetNames(path, file):
    excel_file = f'{path}/{file}'
    wb = xl.load_workbook(excel_file)
    
    if len(wb.sheetnames) == 1:
        return wb.sheetnames[0]
    else:
        return wb.sheetnames


In [4]:
# Carteira Portfólio Cliente
clientCarteira, clientPolitica, _ = getSheetNames(CLIENT_PATH, portfolioCliente)

# Base de Dados dos Fundos
result = getSheetNames(PATH, databaseFundos)

# Histórico Benchmarks
classBenchmarks, classRetornos = getSheetNames(PATH, historicoBenchmarks)


In [5]:
# Carteira do Cliente:
carteira = pd.read_excel(f'{CLIENT_PATH}/{portfolioCliente}', sheet_name=clientCarteira, header=None)
politica = pd.read_excel(f'{CLIENT_PATH}/{portfolioClinete}', sheet_name=clientPolitica, header=None)

# Base de Dados Fundos
database = pd.read_excel(f'{PATH}/{databaseFundos}', sheet_name=result, header=None)

# Árvore Classificação Insper
benchmarks = pd.read_excel(f'{PATH}/{historicoBenchmarks}', sheet_name=classBenchmarks, header=None)
retornosBenchmarks = pd.read_excel(f'{PATH}/{historicoBenchmarks}', sheet_name=classRetornos, header=None)

## Extração de Tabelas e Atribuição de Variáveis Dataframe

#### Análise de Tabelas dentro de cada Excel:

In [6]:
def getTables(df):
    larr = label(np.array(df.notnull()).astype("int"))
    
    list_dfs = []
    
    for s in regionprops(larr):
        sub_df = df.iloc[
            s.bbox[0]:s.bbox[2], s.bbox[1]:s.bbox[3]
        ].pipe(
            lambda df_: df_.rename(columns=df_.iloc[0]).drop(df_.index[0])
        )
    
        list_dfs.append(sub_df)
    return list_dfs


In [7]:
dfsCarteira = getTables(carteira)
dfsPolitica = getTables(politica)
dfsDatabase = getTables(database)
dfsBenchmarks = getTables(benchmarks)
dfsRetornos = getTables(retornosBenchmarks)

#### Criando Variáveis para cada Tabela dos Excels

In [8]:
def attrDataframes(list_dfs):
    def getVarName(df):
        def namestr(obj, namespace):
            return [name for name in namespace if namespace[name] is obj]
    
        return namestr(df, globals())[0]
    
    for i in range(len(list_dfs)):
        dfName = getVarName(list_dfs).replace('dfs', '')
        
        varName = f'df{dfName}{i+1}'
        value = list_dfs[i]
        globals()[varName] = value
        
        print(f'- Dataframe {varName} created.')
    print('')


In [9]:
attrDataframes(dfsCarteira)
attrDataframes(dfsPolitica)
attrDataframes(dfsDatabase)
attrDataframes(dfsBenchmarks)
attrDataframes(dfsRetornos)

- Dataframe dfCarteira1 created.

- Dataframe dfPolitica1 created.
- Dataframe dfPolitica2 created.
- Dataframe dfPolitica3 created.

- Dataframe dfDatabase1 created.

- Dataframe dfBenchmarks1 created.
- Dataframe dfBenchmarks2 created.

- Dataframe dfRetornos1 created.



## Dataframes

### - Carteira do Cliente:

In [10]:
valorCarteira = dfCarteira1["Financeiro"].iloc[-1]
perctCarteira = dfCarteira1["%"].iloc[-1]

In [11]:
dfCarteira1 = dfCarteira1.dropna(subset=["Source"])
dfCarteira1 = dfCarteira1.dropna(subset=["Portfólio"])

In [12]:
dfCarteira1.head()

,Data Posição,Source,Portfólio,Quantidade,Financeiro,Classe,%
5,2023-09-13 00:00:00,KING - BRL,BV CASH FIRF SIMPLES,21170.299483,26943.031989,Renda Fixa Brasil Pós Fixado,0.002191
6,2023-09-13 00:00:00,KING - BRL,BTG PACTUAL DIGITAL TESOURO SELIC SIMPLES FI RF,218395.712765,315562.17117,Renda Fixa Brasil Pós Fixado,0.025664
7,2023-09-13 00:00:00,KING - BRL,LCI 30Jun2023 - 20Jun2025 96.5 BRADESCO,800000,820257.249879,Renda Fixa Brasil Pós Fixado,0.06671
8,2023-09-13 00:00:00,KING - BRL,LCA 30Aug2023 - 19Aug2025 94.5 BRADESCO,1000000,1004178.335827,Renda Fixa Brasil Pós Fixado,0.081668
9,2023-09-13 00:00:00,KING KONG FIM CP IE,LFT01092026,38,264782.350653,Renda Fixa Brasil Pós Fixado,0.021534


### - Política de Investimento

In [13]:
dfPolitica1.head()

,TradingDesk,Book,Min,Max,Estratégico,Tático,Atual
1,KING - BRL,Renda Fixa BR Pós-Fixado,0.05,0.35,0.075,0.13,0.233557
2,KING - BRL,Renda Fixa BR Crédito Privado,0,0.2,0.09,0.09,0.100853
3,KING - BRL,Renda Fixa BR Pré-Fixado,0,0.1,0.04,0,0
4,KING - BRL,Renda Fixa BR Inflação,0,0.15,0.08,0.12,0.109807
5,KING - BRL,Renda Fixa Intl Cash Equivalent,0,0.05,0,0.005,0.004163


In [14]:
dfPolitica2.rename(columns={'%':'Estratégico'}, inplace=True)
dfPolitica2.head()

,Alocação Estratégica por Mercado,Estratégico
23,Renda Fixa,0.315
24,Retorno Absoluto,0.15
25,Renda Variável,0.225
26,Private Equity,0.255
27,Real Estate,0.055


In [15]:
dfPolitica3.head()

,Alocação Estratégica por Geografia,%
31,Brasil,0.575
32,Internacional,0.425


### - Base de Dados Fundos:

In [16]:
dfDatabase1.head()

,ClassName,ClassLevelTwoName,ClassLevelThreeName,Name,InvestmentTypeName,RedemptionQuotationDelayDayCountName,RedemptionQuotationDelayDayCountValue,ExternalCode,ExternalCodeSource
1,Retorno Absoluto,Retorno Absoluto Brasil,Retorno Absoluto Brasil Macro,Ibiuna Hedge STH FIC FIM,InvestmentFund,Dias Corridos,30,IBIUNA HEDGE STH FIC FIM,Lote 45
2,Renda Variável,Renda Variável Doméstico,Renda Variável Doméstico Ações,Bogari Value JC FIC FIA,InvestmentFund,Dias Corridos,30,BOGARI VALE JC FIC FIM,Lote 45
3,Renda Fixa,Renda Fixa Brasil CDI,RF BR CDI Soberano,BTG Pactual Tesouro Selic FI RF REF DI,InvestmentFund,Dias Corridos,0,TESOURO SELIC FI RF,Lote 45
4,Retorno Absoluto,Retorno Absoluto Brasil,Retorno Absoluto Brasil Macro,Kapitalo Zeta FIQ FIM,InvestmentFund,Dias Úteis,60,KAPITALO FIQ ZETA,Lote 45
5,Retorno Absoluto,Retorno Absoluto Brasil,Retorno Absoluto Brasil Equity Hedge,Moat Capital Equity Hedge FIC FIM,InvestmentFund,Dias Úteis,0,MOAT CAPITAL EQUITY HEDGE FIC FIM,Lote 45


### - Base de Dados e Histórico Benchmark:

In [17]:
dfBenchmarks1.head()

,Classes,Benchmark,Ticker Bloomberg
1,Renda Fixa Brasil CDI,CDI,BZACCETP Index
2,Renda Fixa Brasil Crédito Pós-Fixado,IDA-DI,IDADDI Index
3,Renda Fixa Brasil Pré-Fixado,IRF-M 1+,BZRFIR1+ Index
4,Renda Fixa Brasil Inflação,IMA-B,BZRFIMAB INDEX
5,Renda Fixa Internacional Cash Equivalent,ICE U.S. Treasury Short Bond,IDCOTSTR Index


In [19]:
dfRetornos1.head()

,Data,Renda Fixa Brasil CDI,Renda Fixa Brasil Crédito Pós-Fixado,Renda Fixa Brasil Pré-Fixado,Renda Fixa Brasil Inflação,Renda Fixa Internacional Cash Equivalent,Renda Fixa Internacional Pré-Fixado,Renda Fixa Internacional Inflação,Renda Fixa Internacional Crédito,Retorno Absoluto Brasil,Retorno Absoluto Internacional,Renda Variável Brasil,Renda Variável Internacional,Private Equity Brasil,Private Equity Internacional,Real Estate Brasil,Real Estate Internacional,Commodities
1,2012-01-31 00:00:00,0.008874,0.009366,0.014172,0.016315,0.000059,0.009881,0.023482,0.035732,0.016157,0.027792,0.111326,0.050183,0.08251,0,0.030333,0.083108,0.022312
2,2012-02-29 00:00:00,0.007417,0.008443,0.012284,0.021733,-0.000063,0.003615,-0.003757,0.030352,0.015766,0.020579,0.043433,0.048849,0.094339,0,0.046483,0.039845,0.060629
3,2012-03-30 00:00:00,0.00811,0.009322,0.012667,0.019131,0.000087,-0.001447,-0.011348,0.00147,0.012393,-0.001776,-0.019765,0.012856,0.018434,0.0633,0.040653,0.00741,-0.023503
4,2012-04-30 00:00:00,0.007029,0.00821,0.022992,0.044201,0.000153,0.006232,0.021172,0.008794,0.018731,-0.005121,-0.041709,-0.011356,-0.002949,0,0.014343,0.021775,-0.005123
5,2012-05-31 00:00:00,0.007339,0.008355,0.019004,0.01966,0.00019,0.01041,0.018462,-0.02004,0.008606,-0.026106,-0.118567,-0.086324,-0.110974,0,0.034245,-0.067054,-0.129788


## Rebalanceamento do Portfólio:

In [20]:
dfRebalance = dfPolitica1[['Book', 'Min', 'Max', 'Tático', 'Atual']]
dfRebalance

,Book,Min,Max,Tático,Atual
1,Renda Fixa BR Pós-Fixado,0.05,0.35,0.13,0.233557
2,Renda Fixa BR Crédito Privado,0,0.2,0.09,0.100853
3,Renda Fixa BR Pré-Fixado,0,0.1,0,0
4,Renda Fixa BR Inflação,0,0.15,0.12,0.109807
5,Renda Fixa Intl Cash Equivalent,0,0.05,0.005,0.004163
6,Renda Fixa Intl Pré-Fixado,0,0.05,0.01,0.008326
7,Renda Fixa Intl Inflação,0,0.05,0.005,0.002081
8,Renda Fixa Intl Crédito Privado,0,0.1,0.03,0.016028
9,Retorno Absoluto BR,0,0.2,0.1,0.087673
10,Retorno Absoluto Internacional,0,0.1,0.01,0.008326


In [21]:
result = dfPolitica1[['Book', 'Tático', 'Atual']]
result['Offset'] = result['Atual'] - result['Tático']

result = result[['Book', 'Tático', 'Atual', 'Offset']]

result

,Book,Benchmark,Tático,Atual,Offset
1,Renda Fixa BR Pós-Fixado,CDI,0.13,0.233557,0.103557
2,Renda Fixa BR Crédito Privado,IDA-DI,0.09,0.100853,0.010853
3,Renda Fixa BR Pré-Fixado,IRF-M 1+,0,0,0
4,Renda Fixa BR Inflação,IMA-B,0.12,0.109807,-0.010193
5,Renda Fixa Intl Cash Equivalent,ICE U.S. Treasury Short Bond,0.005,0.004163,-0.000837
6,Renda Fixa Intl Pré-Fixado,Bloomberg Global Aggregate,0.01,0.008326,-0.001674
7,Renda Fixa Intl Inflação,Bloomberg US Govt Inflation-Linked,0.005,0.002081,-0.002919
8,Renda Fixa Intl Crédito Privado,Bloomberg Global High Yield,0.03,0.016028,-0.013972
9,Retorno Absoluto BR,IHFA,0.1,0.087673,-0.012327
10,Retorno Absoluto Internacional,HFRI Fund Weighted,0.01,0.008326,-0.001674


### Tracking Error do Portfólio:

### Tracking Error Individual por Aplicação:

### Otimização:

### Resultados

In [22]:

pValue = locale.format_string("%.3f", valorCarteira, grouping=True, monetary=True)
print(f"[Portfolio Value]\n- {pValue}\n")

acoes = []
values = []
for i in range(len(result.index)):
    # print(result['Offset'].iloc[i])
    if result['Offset'].iloc[i] > 0:
        acao = 'Buy' 
    elif result['Offset'].iloc[i] <  0:
        acao = 'Sell'
    else:
        acao = 'Skip'


    # print(acao)

    price = abs(valorCarteira * result['Offset'].iloc[i])
    price = locale.format_string("%.3f", price, grouping=True, monetary=True)
    print(
f'''[{result['Book'].iloc[i]}]
- {acao} {price} of Porfolio Value ({abs(result['Offset'].iloc[i]):.3%})
'''
    )
    
    acoes.append(acao)
    values.append(price)


result['Order'] = acoes
result['Values'] = values

[Portfolio Value]
- 12.295.848,806

[ Renda Fixa BR Pós-Fixado]
- Buy 1.273.319,074 of Porfolio Value (10.356%)

[ Renda Fixa BR Crédito Privado]
- Buy 133.447,909 of Porfolio Value (1.085%)

[ Renda Fixa BR Pré-Fixado]
- Skip 0,000 of Porfolio Value (0.000%)

[ Renda Fixa BR Inflação]
- Sell 125.336,409 of Porfolio Value (1.019%)

[ Renda Fixa Intl Cash Equivalent]
- Sell 10.294,116 of Porfolio Value (0.084%)

[ Renda Fixa Intl Pré-Fixado]
- Sell 20.588,232 of Porfolio Value (0.167%)

[ Renda Fixa Intl Inflação]
- Sell 35.886,680 of Porfolio Value (0.292%)

[ Renda Fixa Intl Crédito Privado]
- Sell 171.803,320 of Porfolio Value (1.397%)

[ Retorno Absoluto BR]
- Sell 151.568,487 of Porfolio Value (1.233%)

[ Retorno Absoluto Internacional]
- Sell 20.588,232 of Porfolio Value (0.167%)

[ Renda Variável BR]
- Buy 166.436,031 of Porfolio Value (1.354%)

[ Renda Variável Internacional]
- Buy 14.336,666 of Porfolio Value (0.117%)

[ Private Equity BR]
- Buy 587.130,877 of Porfolio Value (4

In [26]:
result

# Teste

In [24]:
# importing the openpyxl module as xl 

# excel file used here gfg.xlsx 
excel_file = "./src/Clientes/Portfolio Global - KING.xlsx"

# load the workbook 
wb = xl.load_workbook(excel_file) 

# print the list that stores the sheetnames 
print(wb.sheetnames) 

# we can also get the name of the sheet which is active by using the active property 
# print("Active sheet: ", wb.active) 

# By default the active sheet is the 1st one and can be changed 
wb._active_sheet_index = 4
sheet = wb.active 
# print("Active sheet: ", wb.active) 


['Carteira Global', 'Política de Investimento', 'Suporte']


In [25]:
# teste = pd.read_excel(f'{path}/Clientes/GVB FIM CP IE - CARTEIRA LOCAL.xlsm', sheet_name='Nivel_II', header=None)
# teste